In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('../assets/input/train.csv')
weather = pd.read_csv('../assets/input/weather.csv')

## Climate Cleaning

In [3]:
# Calculate and fill in Tavg using half-up rounding to maintain parity with existing data.
weather['Tavg'] = np.ceil(((weather['Tmax'] + weather['Tmin']) / 2) - .4999).astype(int)

In [4]:
# If Tavg - Depart is the expected temperature for that day, that calculation can
# be used to fill in missing values for Station 2 using data from Station 1.

# In order to run the following list on the Depart series, it is necessary to
# homogenize the datatypes. By replacing the elements that keep it an object
# and casting it as a series of ints, we enable manipulation of the numerics.
weather['Depart'].replace('M', None, inplace=True)
weather['Depart'] = weather['Depart'].astype(int)

# Replace incorrect values for Depart for Station 2 with properly calculated
# values from Station 1 on the same days.
i = 1
while i < len(weather):
    normal = weather['Tavg'].iloc[i - 1] - weather['Depart'].iloc[i - 1]
    weather['Depart'].iloc[i] = weather['Tavg'].iloc[i] - normal
    i += 2
    
# Think of the RAM and dealloc!
del(normal, i)

/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
# WetBulb doesn't have a clear way to calculate missing values, however the
# discrepencies between Station 1 and 2's measurements are within 2 degrees and
# often coincide directly. Since we are already resigned the fact that models
# are not perfect emulations of reality, I don't believe it to be unfair to
# suggest replacing the four missing values with the counterpart station's
# measurement.

# Just as with Depart, it is necessary to first homogenize the datatype of the
# series in order to manipulate it. First we need the index values of the cells
# in question.
ixlist = list(weather[weather['WetBulb'] == 'M'].index.values)

# Replace 'M's with None and typecast as int.
weather['WetBulb'].replace('M', None, inplace=True)
weather['WetBulb'] = weather['WetBulb'].astype(int)

# Using mod, it is easy to ensure the data is being copied from the correct day.
for i in ixlist:
    if i % 2 == 0:
        weather['WetBulb'].iloc[i] = weather['WetBulb'].iloc[i + 1]
    else:
        weather['WetBulb'].iloc[i] = weather['WetBulb'].iloc[i - 1]

# Liberate your memory!
del(ixlist, i)

/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
# Heat and Cool are easily computable given that the base value is given at 65.
# First things first, replace and typecast:
weather['Heat'].replace('M', None, inplace=True)
weather['Cool'].replace('M', None, inplace=True)
weather['Heat'] = weather['Heat'].astype(int)
weather['Cool'] = weather['Cool'].astype(int)

# So it turns out that going through all the rows is more time consuming than
# anticipated. If you made the mistake of running this cell, go ahead and grab
# yourself a coffee.
for i in range(len(weather)):
    if weather['Tavg'].iloc[i] >= 65:
        weather['Heat'].iloc[i] = 0
        weather['Cool'].iloc[i] = weather['Tavg'].iloc[i] - 65
    else:
        weather['Cool'].iloc[i] = 0
        weather['Heat'].iloc[i] = 65 - weather['Tavg'].iloc[i]

# Leave no RAM unnecessarily allocated
del(i)

/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
# Sunrise and sunset are caluclated rather than observed and only missing for
# one station. Its just a simple transcription job. Replace and typecast:
weather['Sunrise'].replace('-', None, inplace = True)
weather['Sunset'].replace('-', None, inplace = True)
weather['Sunrise'] = weather['Sunrise'].astype(int)
weather['Sunset'] = weather['Sunset'].astype(int)

# Much like with Depart, we merely need to alter every other row
i = 1
while i < len(weather):
    weather['Sunrise'].iloc[i] = weather['Sunrise'].iloc[i - 1]
    weather['Sunset'].iloc[i] = weather['Sunset'].iloc[i - 1]
    i += 2

# Every little bit helps
del(i)

/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
# All depth and water values are 0. Since there is no overwriting at play here,
# there is no need to replace and typecast before reassigning.
weather['Depth'] = 0
weather['Water1'] = 0

In [10]:
# So far all the replacements made have been justifiable given the data. In the
# case of Snowfall, slight liberties will be taken. Namely, 'T', for 'trace'
# will be replaced with 0.05 which is half the minimum quantified value in
# order to preserve the observation. Station 2 will have its values set to
# mirror Station 1. To start, whitespaces must be stripped from the set:
weather['SnowFall'] = weather['SnowFall'].str.strip()

#Replace and typecast
weather['SnowFall'].replace('T', .05, inplace=True)
weather['SnowFall'].replace('M', None, inplace=True)
weather['SnowFall'] = weather['SnowFall'].astype(float)

# Mirror Station 1 values for Station 2
i = 1
while i < len(weather):
    weather['SnowFall'].iloc[i] = weather['SnowFall'].iloc[i - 1]
    i += 2

# Dust to dust
del(i)

/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
# Most of the non-numeric values in the PrecipTotal category are 'T' values,
# which we can again replace with half the minimum quantity to preserve the
# measurement. The two 'M' values, can be set to 0 to mirror the other stations
# measurement for that day.
weather['PrecipTotal'] = weather['PrecipTotal'].str.strip()
weather['PrecipTotal'].replace('T', .05, inplace=True)
weather['PrecipTotal'].replace('M', 0.0, inplace=True)
weather['PrecipTotal'] = weather['PrecipTotal'].astype(float)

In [46]:
# StnPressure puts us in an awkward spot. On average, Station 1's pressure is
# .05 to .10 lower than Station 2's, but as far as I know we do not have enough
# information to calculate it precisely. Furthermore on 8/10/13, both stations
# are missing their measurements. To fill in the the 'M' values, we can add or
# subtract in the given range from the other stations value to approximate a
# likely value. For the other 'M' values, using a boolean mask I found what
# other days had similar conditions and will base my numbers off of those.

# Replace and typecast
weather['StnPressure'].replace('M', None, inplace=True)
weather['StnPressure'] = weather['StnPressure'].astype(float)

weather['StnPressure'].iloc[87] = weather['StnPressure'].iloc[86] + .06
weather['StnPressure'].iloc[848] = weather['StnPressure'].iloc[849] - .07
# Value based on the day with the most similar conditions
weather['StnPressure'].iloc[2411] = weather['StnPressure'].iloc[2763]
weather['StnPressure'].iloc[2410] = weather['StnPressure'].iloc[2411] - .06

/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [52]:
# SeaLevel measurements tend to vary between stations by +/- .02. None of the
# missing values occur in contiguous rows, so we can simply set missing values
# equal to the other station measurement which will be good enough.

# Generate list of index values where SeaLevel measurements are missing
ixlist = list(weather[weather['SeaLevel'] == 'M'].index.values)

# Replace 'M's with None and typecast as float.
weather['SeaLevel'].replace('M', None, inplace=True)
weather['SeaLevel'] = weather['SeaLevel'].astype(float)

# Using mod, as we did with WetBulb
for i in ixlist:
    if i % 2 == 0:
        weather['SeaLevel'].iloc[i] = weather['SeaLevel'].iloc[i + 1]
    else:
        weather['SeaLevel'].iloc[i] = weather['SeaLevel'].iloc[i - 1]

# Guillotine the bourgeoisie RAM hoarders
del(ixlist, i)

/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [62]:
# The variance between station measurements of AvgSpeed can be quite large.
# On the plus side, there are only three missing values in this category. To
# generate predictions for these missing values, we can measure the difference
# in ResultSpeed between the two stations and apply that difference to the
# measurement of AvgSpeed and call it good enough.

ixlist = list(weather[weather['AvgSpeed'] == 'M'].index.values)

# Replace 'M's with None and typecast as float.
weather['AvgSpeed'].replace('M', None, inplace=True)
weather['AvgSpeed'] = weather['AvgSpeed'].astype(float)

# All missing values are for Station 2 measurements.
for i in ixlist:
    diff = weather['ResultSpeed'].iloc[i - 1] - weather['ResultSpeed'].iloc[i]
    weather['AvgSpeed'].iloc[i] = weather['AvgSpeed'].iloc[i - 1] - diff

# The new Purge film continues the trend of being less compelling than the last
del(ixlist, diff, i)

/Users/Arya/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [150]:
# Mapping the codes in CodeSum into binary categorical columns.
weather['Thunderstorm'] = np.where(weather[['CodeSum']].apply(
    lambda x: x.str.contains(r'TS')).any(1), '1', '0')
weather['Rain'] = np.where(weather[['CodeSum']].apply(
    lambda x: x.str.contains(r'RA')).any(1), '1', '0')
weather['Drizzle'] = np.where(weather[['CodeSum']].apply(
    lambda x: x.str.contains(r'DZ')).any(1), '1', '0')
weather['Snow'] = np.where(weather[['CodeSum']].apply(
    lambda x: x.str.contains(r'SN')).any(1), '1', '0')
weather['Fog'] = np.where(weather[['CodeSum']].apply(
    lambda x: x.str.contains(r'FG')).any(1), '1', '0')
weather['Mist'] = np.where(weather[['CodeSum']].apply(
    lambda x: x.str.contains(r'BR')).any(1), '1', '0')
weather['Haze'] = np.where(weather[['CodeSum']].apply(
    lambda x: x.str.contains(r'HZ')).any(1), '1', '0')
weather['Smoke'] = np.where(weather[['CodeSum']].apply(
    lambda x: x.str.contains(r'FU')).any(1), '1', '0')

# Typecast newly created encoded columns as numeric
weather[['Thunderstorm', 'Rain', 'Drizzle', 'Snow', 'Fog', 'Mist', 'Haze',
         'Smoke']] = weather[['Thunderstorm', 'Rain', 'Drizzle', 'Snow', 'Fog',
                              'Mist', 'Haze', 'Smoke']].apply(pd.to_numeric)

In [163]:
# Export to csv
weather.to_csv('../assets/input/clean_weather.csv')